# Tutorial on how to use mescal
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [1]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
# Set up your Brightway project
bd.projects.set_current('ei3.8-mescal') # put the name of your brightway project here

## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [3]:
mapping = pd.read_csv('../dev/energyscope_data/mapping.csv')
unit_conversion = pd.read_csv('../dev/energyscope_data/unit_conversion.csv')
mapping_esm_flows_to_CPC = pd.read_csv('../dev/energyscope_data/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv('../dev/energyscope_data/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', or 'Resource')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the region of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

In [4]:
mapping.head()

,Name,Type,Product,Activity,Location,Unit,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,/kg,h2_electrolysis
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,/unit,h2_electrolysis
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,/unit,h2_electrolysis
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,/unit,h2_electrolysis
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,/unit,h2_electrolysis


A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', or 'Resource')
- `Value`: the numerical value of the conversion factor

In [5]:
unit_conversion.head()

,Name,Type,Value
0,ACETIC_ACID,Resource,247422.680400
1,ACETONE,Resource,121654.501200
2,AEC_OG,Construction,1555.555556
3,AEC_OG,Operation,30030.030030
4,AEC_OG_PLANT,Construction,1.000000


A mapping between the energy model flows and CPC categories. The list of products should at least encompass the list of technologies without a CPC category within your mapping and their foreground inventory. The mapping should be provided in a dataframe with the following columns:
- `Product`: the name of the product in the LCI database
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [6]:
mapping_esm_flows_to_CPC.head()

,Flow,Description,CPC
0,BENZENE,Benzene,"['33100: Coke and semi-coke of coal, of lignit..."
1,BIO_DIESEL,Bio-diesel,['35491: Biodiesel']
2,CO2_A,Carbon dioxide (concentrated emissions),['34210b: Carbon dioxide and monoxide']
3,CO2_C,Carbon dioxide (captured),['34210b: Carbon dioxide and monoxide']
4,CO2_CS,Carbon dioxide (captured & stored),['34210b: Carbon dioxide and monoxide']


The input and output flows if energy technologies. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [7]:
model.head()

,Name,Flow,Amount
0,DIESEL_S,DIESEL_S,1.0
1,GASOLINE_S,GASOLINE_S,1.0
2,ELEC_S,ELEC_S,1.0
3,NG_S,NG_S,1.0
4,H2_S,H2_S,1.0


### Optional dataframes

In [4]:
technology_compositions = pd.read_csv('../dev/energyscope_data/technology_compositions.csv')
technology_specifics = pd.read_csv('../dev/energyscope_data/technology_specifics.csv') 
lifetime = pd.read_csv('../dev/energyscope_data/lifetime.csv')
mapping_product_to_CPC = pd.read_csv('../dev/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/IW+/impact_abbrev.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [9]:
technology_compositions.head()

,Name,Components
0,AEC_OG,"['AEC_OG_STACK', 'AEC_OG_PLANT', 'AEC_OG_STACK..."
1,ALKALINE_ELECTROLYSIS,"['ALKALINE_ELECTROLYSIS_STACK', 'ALKALINE_ELEC..."
2,AN_DIG_SI,"['AN_DIG_SI_PLANT', 'AN_DIG_SI_COGEN']"
3,ATR,"['ATR_PLANT', 'ATR_TANK']"
4,ATR_CCS,"['ATR_CCS_PLANT', 'ATR_CCS_TANK']"


A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bioprocesses (if mismatch fuel in the LCI dataset). The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the list of requirements  
- `Amount`: the numerical value of the requirement (if relevant)

In [10]:
technology_specifics.head()

,Name,Specifics,Amount
0,CO2_METHANOL,Process,NaN
1,ETHANOL_TO_JETFUELS,Process,NaN
2,ETHANE_OXIDATION,Process,NaN
3,METHANOL_TO_AROMATICS,Process,NaN
4,METHANOL_TO_OLEFINS,Process,NaN


Energy technologies lifetimes in the ESM and the LCI database. The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)

In [11]:
lifetime.head()

,Name,ESM,LCA
0,AEC_OG,20.0,NaN
1,AEC_OG_PLANT,NaN,20.0
2,AEC_OG_STACK,NaN,7.5
3,AEC_OG_PLANT_DECOM,NaN,20.0
4,AEC_OG_STACK_DECOM,NaN,7.5


In case you have a LCI database without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the (partial) name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: whether the `Name` entry is an exactly the name to look for, or is contained in the full name
- `Where`: whether the `Name` entry is meant for products or activities

In [12]:
mapping_product_to_CPC.head()

,Name,CPC,Search type,Where
0,amine-based silica,"35310: Organic surface active agents, except soap",equals,Product
1,biodiesel,35491: Biodiesel,contains,Product
2,"biomass, used as fuel",31230: Wood in chips or particles,equals,Product
3,"biomethane, from biogas upgrading, using amine...","12020: Natural gas, liquefied or in the gaseou...",equals,Product
4,"biomethane, high pressure","12020: Natural gas, liquefied or in the gaseou...",equals,Product


An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category

In [13]:
impact_abbrev.head()

,Impact_category,Unit,Abbrev,AoP
0,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQL,EQ
1,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQS,EQ
2,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQLB,EQ
3,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQSB,EQ
4,"('IMPACT World+ Damage 2.0.1', 'Human health',...",DALY,CCHHL,HH


## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [5]:
name_premise_db = "ecoinvent_cutoff_3.8_remind_SSP2-Base_2020"
name_premise_comp_db = name_premise_db + "_comp_QC"

In [6]:
name_premise_with_CPC_db = name_premise_db + '_with_CPC'
name_premise_comp_with_CPC_db = name_premise_comp_db + '_with_CPC'

In [7]:
premise_db = load_extract_db(name_premise_db)
premise_comp_db = load_extract_db(name_premise_comp_db)

In [24]:
create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:51


Title: Writing activities to SQLite3 database:
  Started: 06/21/2024 10:27:53
  Finished: 06/21/2024 10:28:45
  Total time elapsed: 00:00:51
  CPU %: 60.40
  Memory %: 18.13


In [25]:
create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_comp_with_CPC_db, mapping_product_to_CPC=mapping_product_to_CPC)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/21/2024 10:32:46
  Finished: 06/21/2024 10:32:46
  Total time elapsed: 00:00:00
  CPU %: 20.00
  Memory %: 18.12


In [26]:
premise_comp_db = [] # free memory
premise_comp_db_with_CPC = load_extract_db(name_premise_comp_with_CPC_db)

Getting activity data


100%|██████████| 37/37 [00:00<00:00, 11456.46it/s]


Adding exchange data to activities


100%|██████████| 1170/1170 [00:01<00:00, 1134.47it/s]


Filling out exchange data


100%|██████████| 37/37 [00:00<00:00, 134.00it/s]


In [27]:
relink_database(premise_comp_db_with_CPC, name_premise_db, name_premise_with_CPC_db)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/21/2024 10:32:48
  Finished: 06/21/2024 10:32:49
  Total time elapsed: 00:00:00
  CPU %: 31.30
  Memory %: 18.12


## Operations on the mapping dataframe (optional)

In [8]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db)

### Relink the mapping dataframe with another database
In this example, we relink the current mapping, which is based on ecoinvent and some additional inventories, with a [premise](https://premise.readthedocs.io/en/latest/introduction.html) database. The function `create_complementary_database` takes as input the mapping dataframe, the database to link to, and the name of a newly created complementary database, in case some LCI datasets are missing in the LCI database we relink to. It returns a new mapping dataframe with the location column updated, and create the complementary database in the brightway project.

In [9]:
mapping_linked_to_premise = create_complementary_database(mapping, premise_db_with_CPC, name_premise_comp_with_CPC_db)

No inventory in the premise database for ('EHP_H2_GRID', 'Construction')
No inventory in the premise database for ('HP_H2_GRID', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B100_MD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B100_MD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B100_SD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B100_SD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B20_MD', 'Construction')
No inventory in the premise database for ('LCV_BIODIESEL_B20_MD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B20_SD', 'Operation')
No inventory in the premise database for ('LCV_BIODIESEL_B20_SD', 'Construction')
No inventory in the premise database for ('LCV_CNG_MD', 'Construction')
No inventory in the premise database for ('LCV_CNG_MD', 'Operation')
No inventory in the premise database for ('LCV_CNG_SD', 'Construct

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/21/2024 11:11:56
  Finished: 06/21/2024 11:11:56
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 23.46


In [10]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...


### Add or replace the location column based on a user-defined ranking
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [11]:
# create a concatenated database of all databases in the mapping dataframe (including background requirements)
base_db = concatenate_databases(list(mapping_linked_to_premise.Database.unique()))

Getting activity data


100%|██████████| 37/37 [00:00<00:00, 15087.42it/s]


Adding exchange data to activities


100%|██████████| 1170/1170 [00:00<00:00, 74153.58it/s]


Filling out exchange data


100%|██████████| 37/37 [00:00<00:00, 180.18it/s]


In [12]:
# Define the user-defined ranking
my_ranking = [
    'CA-QC', # Quebec
    'CA', # Canada
    'CA-ON', # Other canadian provinces 
    'CA-AB',
    'CA-BC',
    'CA-MB',
    'CA-NB',
    'CA-NF',
    'CA-NS',
    'CA-NT',
    'CA-NU',
    'CA-PE',
    'CAZ', # Canada - Australia - New Zealand
    'RNA', # North America
    'US', # United States
    'USA', # United States
    'GLO', # Global average 
    'RoW', # Rest of the world
]

In [13]:
# Update mapping dataframe
mapping_linked_to_premise = change_location_mapping_file(mapping_linked_to_premise, my_ranking, base_db, 'QC')

In [14]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AEC_OG,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
1,AEC_OG_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
2,AEC_OG_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
3,AEC_OG_STACK,Construction,"electrolyzer, 1MWe, AEC, Stack","electrolyzer production, 1MWe, AEC, Stack",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...
4,AEC_OG_STACK_DECOM,Construction,"used fuel cell stack, 1MWe, AEC","treatment of fuel cell stack, 1MWe, AEC",RER,ecoinvent_cutoff_3.8_remind_SSP2-Base_2020_wit...


In [15]:
# Save the relinked mapping file
mapping_linked_to_premise.to_csv('../dev/energyscope_data/mapping_linked.csv', index=False)

## Double-counting removal

In [6]:
# To skip the previous steps
# mapping_linked_to_premise = pd.read_csv('../dev/energyscope_data/mapping_linked.csv')

In [18]:
new_db_name = 'energyscope_QC_2020'

In [19]:
regionalize_foregrounds = True

In [20]:
mismatch_regions = ['GLO', 'RoW', 'RER', 'CH']  # insufficient match within ecoinvent

In [21]:
premise_comp_db_with_CPC = [] # free memory
base_db = concatenate_databases(list(mapping_linked_to_premise.Database.unique()))

Getting activity data


100%|██████████| 30201/30201 [00:00<00:00, 37961.30it/s] 


Adding exchange data to activities


100%|██████████| 1022813/1022813 [00:40<00:00, 24965.14it/s]


Filling out exchange data


...

100%|██████████| 37/37 [00:00<00:00, 36949.82it/s]


Adding exchange data to activities


100%|██████████| 1170/1170 [00:00<00:00, 41788.03it/s]


Filling out exchange data


100%|██████████| 37/37 [00:00<00:00, 195.55it/s]


In [22]:
mapping_linked_to_premise_new_code = create_esm_database(
    mapping=mapping_linked_to_premise,
    model=model,
    tech_specifics=technology_specifics,
    technology_compositions=technology_compositions,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    main_database=base_db,
    esm_db_name=new_db_name,
    regionalize_foregrounds=regionalize_foregrounds,
    mismatch_regions=mismatch_regions,
    target_region='CA-QC',
    locations_ranking=my_ranking
)

AEC_OG
ALKALINE_ELECTROLYSIS
AL_MAKING
AL_MAKING_HR
AN_DIG
AN_DIG_SI
ATR
ATR_CCS
BIOGAS_ATR
BIOGAS_ATR_CCS
BIOGAS_BIOMETHANE
BIOGAS_SMR
BIOGAS_SMR_CCS
BIOMASS_ETHANOL
BIOMASS_GAS_EF_H2
BIOMASS_GAS_EF_H2_CCS
BIOMASS_GAS_FB_H2
BIOMASS_GAS_FB_H2_CCS
BUS_BIODIESEL_B100_SD
BUS_BIODIESEL_B20_SD
BUS_CNG_SD
BUS_DIESEL_SD
BUS_EV_SD
BUS_FC_CH4_SD
BUS_FC_H2_SD
BUS_HY_DIESEL_SD
BUS_PROPANE_SD
CAR_BIODIESEL_B100_ELD
CAR_BIODIESEL_B100_LD
CAR_BIODIESEL_B100_MD
CAR_BIODIESEL_B100_SD
CAR_BIODIESEL_B20_ELD
CAR_BIODIESEL_B20_LD
CAR_BIODIESEL_B20_MD
CAR_BIODIESEL_B20_SD
CAR_CNG_ELD
CAR_CNG_LD
CAR_CNG_MD
CAR_CNG_SD
CAR_DIESEL_ELD
CAR_DIESEL_LD
CAR_DIESEL_MD
CAR_DIESEL_SD
CAR_ETOH_E10_ELD
CAR_ETOH_E10_LD
CAR_ETOH_E10_MD
CAR_ETOH_E10_SD
CAR_ETOH_E85_ELD
CAR_ETOH_E85_LD
CAR_ETOH_E85_MD
CAR_ETOH_E85_SD
CAR_EV_ELD
CAR_EV_LD
CAR_EV_MD
CAR_EV_SD
CAR_FC_CH4_ELD
CAR_FC_CH4_LD
CAR_FC_CH4_MD
CAR_FC_CH4_SD
CAR_FC_H2_ELD
CAR_FC_H2_LD
CAR_FC_H2_MD
CAR_FC_H2_SD
CAR_GASOLINE_ELD
CAR_GASOLINE_LD
CAR_GASOLINE_MD
CAR_GASOLI

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 06/20/2024 10:19:08
  Finished: 06/20/2024 10:19:09
  Total time elapsed: 00:00:01
  CPU %: 23.30
  Memory %: 31.85


## Computing the LCA metrics 

In [28]:
base_db = [] # free memory
esm_db = load_extract_db(new_db_name, create_pickle=False)

Getting activity data


100%|██████████| 1543/1543 [00:00<00:00, 73461.50it/s]


Adding exchange data to activities


100%|██████████| 52318/52318 [00:02<00:00, 25755.23it/s]


Filling out exchange data


100%|██████████| 1543/1543 [00:02<00:00, 632.55it/s] 


In [29]:
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping_linked_to_premise_new_code,
    technology_compositions=technology_compositions,
    methods=['IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1'],
    unit_conversion=unit_conversion,
    lifetime=lifetime
)

In [30]:
R_long.head()

,Impact_category,New_code,Value,Name,Type
0,"(IMPACT World+ Damage 2.0.1, Ecosystem quality...",h55rn8asb8337q12ml08tok4at9gq65u,3.003785e+09,AFC,Construction
1,"(IMPACT World+ Damage 2.0.1, Ecosystem quality...",h55rn8asb8337q12ml08tok4at9gq65u,9.979548e+08,AFC,Construction
2,"(IMPACT World+ Damage 2.0.1, Human health, Cli...",h55rn8asb8337q12ml08tok4at9gq65u,1.366992e+04,AFC,Construction
3,"(IMPACT World+ Damage 2.0.1, Human health, Cli...",h55rn8asb8337q12ml08tok4at9gq65u,4.611984e+03,AFC,Construction
4,"(IMPACT World+ Midpoint 2.0.1, Midpoint, Clima...",h55rn8asb8337q12ml08tok4at9gq65u,5.239713e+09,AFC,Construction


In [31]:
R_long.to_csv('results/impact_scores.csv', index=False)

## Convert the results in an AMPL format

In [5]:
# To skip the previous steps
# R_long = pd.read_csv('results/impact_scores.csv')

In [6]:
refactor = 1e-2
lcia_method = 'IMPACT World+ Damage 2.0.1 - Total only'

### Create the .dat file

In [7]:
normalize_lca_metrics(
    R=R_long,
    f_norm=1e6,
    mip_gap=1e-6,
    refactor=refactor,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev
)

### Create the .mod file

In [8]:
gen_lcia_obj(
    lcia_method=lcia_method,
    refactor=refactor,
    impact_abbrev=impact_abbrev
)